In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
:parameter
--------------------
data: n-array like
[
[1.0 2.3 2.1 0.2]
[0.0 2.4 1.1 0.7]
]
每个元素都是有表示距离的属性的，对于没有距离属性的要抓化成具有距离属性的
"""

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d import Axes3D
from geopy.distance import geodesic


dir_name = '/Users/changqingli/data/time_space_cluster/'
feat_file_mini = 'fs_mini'
feat_file_all = 'fs_all'  # time,lant,long,geohash,buname
feat_file_user_shanghai_mini = 'fs_user_shanghai_mini'
feat_file_user_shanghai_all = 'fs_user_shanghai_all'  # userid,time,lant,long,geohash,buname
feat_file_user_consume_shanghai_mini = 'fs_user_consume_mini'  # 32398964 2019-09-13 12:49:40 31.226408 121.49096 到餐 春风松月楼素菜馆 dining 城隍庙/豫园 1


def key_count(cluster_y):
    m = {}
    for i in cluster_y:
        if i not in m:
            m.setdefault(i, 0)
        m[i] += 1
    res = sorted(m.items(), key=lambda x: x[1], reverse=True)
    return res



In [2]:
f_data = []
user_info = {}
hour_info = {}
user_base = []
consume_poi_type = {}
consume_poi_type_time = {}
consume_poi_lbs = []

with open(dir_name + feat_file_user_consume_shanghai_mini, 'r') as f:
    for line in f:
        line = line.strip().split("\t")
        if float(line[2])<30.40 or float(line[2])>31.53 or float(line[3])<120.52 or float(line[3])>122.12:
            continue
        line.insert(2, line[1][:4])  # _year
        line.insert(2, line[1][5:7])  # _month
        line.insert(2, line[1][8:10])  # _date
        line.insert(2, line[1][11:13])  # _hour
        
        if line[2] not in consume_poi_type:
            consume_poi_type[line[2]] = {}
        if line[-3] not in consume_poi_type[line[2]]:
            consume_poi_type[line[2]][line[-3]] = 0
        consume_poi_type[line[2]][line[-3]] += 1
        
        if line[-3] not in consume_poi_type_time:
            consume_poi_type_time[line[-3]] = {}
        if line[2] not in consume_poi_type_time[line[-3]]:
            consume_poi_type_time[line[-3]][line[2]] = 0
        consume_poi_type_time[line[-3]][line[2]] += 1

        tmp = [line[2]]
        tmp.extend(line[6:8])
        consume_poi_lbs.append(tmp)
        


In [17]:
len(consume_poi_lbs)


24763

In [4]:
# f_data_tmp = consume_poi_lbs
f_data_tmp = np.array(consume_poi_lbs)
f_data = f_data_tmp
scale = MinMaxScaler()
scale.fit(f_data)
f_data_transform = scale.transform(f_data)




In [5]:
k_means = KMeans()
k_means.fit(f_data_transform[1:])
cluster_train_counts = key_count(k_means.labels_)
print(cluster_train_counts)



[(6, 6198), (2, 4867), (3, 4197), (0, 3865), (7, 2318), (4, 1842), (5, 924), (1, 551)]


In [6]:
test_label = np.reshape(k_means.labels_, [len(k_means.labels_), -1])




In [16]:
np.shape(f_data_transform[:, 1:])


(24763, 2)

In [9]:
# cluster_test = np.concatenate((np.reshape(f_data_tmp[:, 0], [len(f_data_tmp), 1]), f_data_transform[:, 4:], test_label.reshape([len(test_label), 1])), axis=1)
# df=pd.DataFrame(cluster_test, columns=['userid', 'f_0', 'f_1', 'label']) #转换数据格式

cluster_test = np.concatenate((f_data_transform[:, 1:], test_label.reshape([len(test_label), 1])), axis=1)
df=pd.DataFrame(cluster_test, columns=['f_0', 'f_1', 'label']) #转换数据格式


# df['userid'] = df['userid'].astype("int")
df['f_0'] = df['f_0'].astype("float")
df['f_1'] = df['f_1'].astype("float")
df['label'] = df['label'].astype("int")


df.dtypes



ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
df.head()

In [ ]:

colr = ['#E15759', '#4E79A7', '#76B7B2', '#F28E2B','blue','#F45E2B','#F67E2B', 'r']

plt.figure(figsize=(7, 7), dpi=80)
axes = plt.subplot(111)
types = []
nums = []
for i in range(8):
    ii = str(i)
    d=df[df['label']==i]
    print("label %d has %d sample" % (i, len(d['f_1'].values)))
    type1 = axes.scatter(d['f_0'].values, d['f_1'].values, s=20, c=colr[i])
    types.append(type1)
    nums.append(str(i) + '_' + str(len(d['f_1'].values)))
axes.legend(types, nums, loc=2)

d=df[df['label']==2]
 

In [ ]:
import geopandas as gp